In [3]:
import pandas as pd

In [4]:
benefits = pd.read_csv("stg/benefits.csv")
entities = pd.read_csv("stg/entities.csv")

Vamos a clasificar los subvencionados por tipo (persona, empresa...) y subtipo (SL, SA, Asociación...)

# Granted Benefits

In [5]:
benefits["tipo"] = ''
benefits["subtipo"] = ''
benefits.loc[benefits['beneficiary_id'] == '*********', ["tipo", 'subtipo']] = ['NO ID'] * 2

def company_person_discerner(id: str) -> str:
    first_position = id[0]
    if first_position.isalpha() and first_position not in ['K', 'L', 'M', 'X', 'Y', 'Z']:
            return "COMPANY"
    else:
        return "PERSON"

benefits.loc[benefits['tipo'] != 'NO ID', 'tipo'] = benefits.query("tipo != 'NO ID'")['beneficiary_id'].apply(company_person_discerner)

def get_person_subtype(id_person: str) -> str:
    if id_person[0] in ['K', 'L', 'M', 'X', 'Y', 'Z']:
        return "EXTRANJERO"
    else:
        return "CITIZEN" 
    
benefits.loc[benefits['tipo'] == 'PERSON', 'subtipo'] = benefits.query("tipo == 'PERSON'")['beneficiary_id'].apply(get_person_subtype)

def get_company_subtype(id_person: str) -> str:
    first_letter = id_person[0]
    
    company_type_dict = {
        'A': 'Sociedades anónimas',
        'B': 'Sociedades de responsabilidad limitada',
        'C': 'Sociedades colectivas',
        'D': 'Sociedades comanditarias',
        'E': 'Comunidades de bienes',
        'F': 'Sociedades cooperativas',
        'G': 'Asociaciones y Fundaciones',
        'H': 'Comunidades de propietarios en régimen de propiedad horizontal',
        'J': 'Sociedades civiles, con o sin personalidad jurídica',
        'N': 'Entidades extranjeras',
        'P': 'Corporaciones Locales',
        'Q': 'Organismos públicos',
        'R': 'Congregaciones e instituciones religiosas',
        'S': 'Órganos de la Administración General del Estado y de las comunidades autónomas',
        'U': 'Uniones Temporales de Empresas',
        'V': 'Otros tipos no definidos en el resto de claves',
        'W': 'Establecimientos permanentes de entidades no residentes en España'
    }
    
    return str.upper(company_type_dict[first_letter])

benefits.loc[benefits['tipo'] == 'COMPANY', 'subtipo'] = benefits.query("tipo == 'COMPANY'")['beneficiary_id'].apply(get_company_subtype)

In [6]:
# benefits.query("tipo=='COMPANY'").groupby('subtipo').subtipo.count().sort_values(ascending=False).plot.bar()

In [28]:
def clean_sa(name: str) -> str:
    if name.endswith(' SA'):
        name = name[:-3]
    elif name.endswith(', SA'):
        name = name[:-4]
    elif name.endswith(', S.A.'):
        name = name[:-6]
    elif name.endswith(' S.A.'):
        name = name[:-5]
    elif name.endswith(' S.A'):
        name = name[:-4]
    elif name.endswith(' S. A.'):
        name = name[:-6]
    elif name.endswith('. SAL'):  # sociedad anónima laboral 
        name = name[:5]
    name = name.replace(' SOCIEDAD ANONIMA', '')
    return name

def clean_sl(name: str) -> str:
    if name.endswith(', SL'):
        name = name[:-4]
    elif name.endswith(' SL'):
        name = name[:-3]
    elif name.endswith(', S.L'):
        name = name[:-5]
    elif name.endswith(', S.L.'):
        name = name[:-6]  
    elif name.endswith(' S. L.'):
        name = name[:-6]
    elif name.endswith(' S.L.'):
        name = name[:-5]
    elif name.endswith(', SLU'):
        name = name[:-5]
    elif name.endswith(', S.L.U.'):
        name = name[:-8]
    elif name.endswith(' S.L.U.'):
        name = name[:7]
    elif name.endswith(' SLL'):  # sociedad limitada laboral 
        name = name[:4]
    name = name.replace(' SOCIEDAD LIMITADA', '')
    return name

def clean_sociedad_civil(name: str) -> str:
    if name.endswith(', S.C.'):
        name = name[:-6]
    elif name.endswith(' S.C.'):
        name = name[:-5]
    elif name.endswith(', SC'):
        name = name[:-4]
    elif name.endswith(' SC.'):
        name = name[:-4]
    elif name.endswith(' SC'):
        name = name[:-3]
    elif name.endswith(', SC - OCM VINO INVER'):
        name = name.replace(', SC - OCM VINO INVER', '')
    elif name.endswith(' SC - OCM VINO INVER'):
        name = name.replace(' SC - OCM VINO INVER', '')
    elif name.endswith(' SL - OCM VINO INVER'):
        name = name.replace(' SL - OCM VINO INVER', '')
    elif name.endswith(' SL -  OCM VINO INVER'):
        name = name.replace(' SL -  OCM VINO INVER', '')
    elif name.endswith(', SC-OCM VINO PROMO'):
        name = name.replace(', SC-OCM VINO PROMO', '')
    name = name.replace(' SOCIEDAD CIVIL', '')
    return name
benefits.loc[benefits['subtipo']=='SOCIEDADES DE RESPONSABILIDAD LIMITADA', 'beneficiary_name'] = benefits.query("subtipo=='SOCIEDADES DE RESPONSABILIDAD LIMITADA'").beneficiary_name.apply(clean_sl)
benefits.loc[benefits['subtipo']=='SOCIEDADES ANÓNIMAS', 'beneficiary_name'] = benefits.query("subtipo=='SOCIEDADES ANÓNIMAS'").beneficiary_name.apply(clean_sa)
benefits.loc[benefits['subtipo']=='SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JURÍDICA', 'beneficiary_name'] = benefits.query("subtipo=='SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JURÍDICA'").beneficiary_name.apply(clean_sociedad_civil)

In [30]:
# Tratar las OCM VINO INVER (https://www.euskadi.eus/ayuda_subvencion/2021/ocm-inversion-2022/web01-tramite/es/)
# Puede que interese guardar lo de la derecha del guión (investigar casos como el de OCM VINO INVER)
benefits.query("subtipo=='SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JURÍDICA' and beneficiary_name.str.contains('SC')").head(20)

,beneficiary_id,beneficiary_name,tipo,subtipo
5310,J01193242,PASCUAL BERGANZO,COMPANY,"SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JUR..."
16296,J95657110,FRANCISCO JAVIER MORO PEREZ-OSCAR MAZA GUZMAN,COMPANY,"SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JUR..."
16368,J01521194,MASTERCLASS ENGLISH SCHOOL,COMPANY,"SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JUR..."
16684,J48833693,JOSE RAMON HERNAIZ GOMEZ- MARIA ASCENSION FERN...,COMPANY,"SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JUR..."
25394,J01292119,PESCADOS ANAIAK,COMPANY,"SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JUR..."
29655,J95850376,FRANCISCO JAVIER LOPEZ FAILDE - VERONICA ARAND...,COMPANY,"SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JUR..."
30809,J95840773,GABINO MADRAZO OROZCO-ANA MARIA DEIROS FOMBELL...,COMPANY,"SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JUR..."
31825,J95862090,FRANCISCO JAVIER GUTIERREZ COLINA-VIVIANA RAQU...,COMPANY,"SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JUR..."
33559,J01178508,PISCIS,COMPANY,"SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JUR..."
34368,J95830915,NERE ASPIAZU BASCARAN-ION RZ. DE EGUINO AZCARA...,COMPANY,"SOCIEDADES CIVILES, CON O SIN PERSONALIDAD JUR..."


# Directorio

In [ ]:
from thefuzz import fuzz

/home/mol/Escritorio/Proyectos/AnalisisSubvenciones/venv/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
np_entities = entities.query("type=='ENTITY'")

In [ ]:
entities[['type', 'subtype']].value_counts()

In [ ]:
# TODO: Limpiar los "sl", ", sl", ", s.l.", "sll" y ese tipo de cosas al final de los nombres.
# Quitar donde haya más de un espacio
# Se tienen que eliminar palabras como comunicación, comunicaciones y ese estilo para evitar los mapeos mal hechos

In [52]:
def normalize_name(x: str) -> str:
    tilde_dict = {
        'á': 'a',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'u': 'u'
    }
    for key in tilde_dict:
        x = x.lower().replace(key, tilde_dict[key])
    return x

entity_names = np_entities.query("subtype=='COMPANY'").name.apply(normalize_name).unique().tolist()
benefited_names = benefits.query("tipo=='COMPANY'").beneficiary_name.apply(normalize_name).unique().tolist()


In [53]:
entities[['type', 'subtype']].value_counts()

type       subtype         
PERSON     GENERAL             2960
           ACCOUNTABLE          697
ENTITY     PUBLIC_SECTOR        695
           COMPANY              521
EQUIPMENT  GENERAL              289
ENTITY     GENERAL              220
           FOREIGN_AFF          204
           ASSOCIATION          130
           PROF_ASSOCIATION     104
           FOUNDATION            24
           LEGISLATIVE           22
           RELIGIOUS              9
           JUDICIAL               8
           POLITICAL_PARTY        5
dtype: int64

In [56]:
from tqdm import tqdm


i = 0
for entity in tqdm(entity_names):
    i += 1
    if '20' not in entity:
        continue
    if i % 10 == 0:
        print(i, '/', len(entity_names))
    for ben_name  in benefited_names:
        similarity = fuzz.ratio(entity, ben_name)
        if similarity > 70:
            print(entity, '< --- >', ben_name, '< --- >', similarity)


 36%|███▋      | 189/521 [00:11<00:22, 14.49it/s]

mediasal 2000, s.a. < --- > instalmyr 2000, s.l. < --- > 72
mediasal 2000, s.a. < --- > dinitel 2000 s.a. < --- > 72
mediasal 2000, s.a. < --- > rodoia 2000, s.l. < --- > 72
mediasal 2000, s.a. < --- > arbeis 2000, s.l. < --- > 72
mediasal 2000, s.a. < --- > movitel 2000, s.a.l. < --- > 72
mediasal 2000, s.a. < --- > cemevisa 2000 sa < --- > 74
mediasal 2000, s.a. < --- > peninsula 2000, s.a. < --- > 72


 75%|███████▌  | 392/521 [00:13<00:03, 33.64it/s]

20 comunicacion < --- > 20 comunicacion, s.l. < --- > 83


100%|██████████| 521/521 [00:16<00:00, 32.46it/s]


In [39]:
fuzz.ratio('colegio oficial de ingenieros de minas y energia del pais vasco, navarra, la rioja y soria', 'ceesa s.a.')

14